# DREAM TEAM for STARTUP IDEAS
Use this tool to generate team assignments for Startup Ideas.

## Setup
- install Python3
- create a virtualenv: `% python3 -m venv studio-env`
- activate the virtualenv: `% source studio-env/bin/activate`
- install the packages: `% pip install -r requirements.txt`
- launch the Jupyter Lab: `% jupyter lab`
- load this notebook

In [ ]:
import csv
from io import StringIO
from ortools.linear_solver import pywraplp

This is the input file, with student names and sections.

In [ ]:
STUDENT_CSV = "startup-ideas-2018.csv"

In [ ]:
ALL_STUDENTS = []
SKIP_SECTION_MIX = False
SKIP_MBA_MIX = False
SECTIONS = ['Section 1', 'Section 2', 'Section 3']

csv_reader = csv.reader(open(STUDENT_CSV))
next(csv_reader)
for student in csv_reader:
    (last_name, first_name, email, program, section) = student
    ALL_STUDENTS.append({ 'section': section, 'id': email, 'first_name': first_name, 'last_name': last_name, 'program': program })

print("Total number of students: %d." % len(ALL_STUDENTS))
for section in SECTIONS:
    students_in_section = [s for s in ALL_STUDENTS if s['section'] == section]
    print("%d student(s) in %s." % (len(students_in_section), section))

In [ ]:
import math
import random

In [ ]:
# Method to generate the teams.
def create_teams(all_students, min_team_size, max_team_size):
    number_of_teams = math.floor(len(all_students) / min_team_size)
    all_teams = [ 'Team%03d' % k for k in range (1, number_of_teams+1)]
    
    solver = pywraplp.Solver('StudentProjectGridCBC', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    matches = {}
    for student in all_students:
        for team in all_teams:
            matches[student['id'], team] = solver.IntVar(0, 1, 'matches[%s,%s]' % (student['id'], team))

    z = solver.Sum(  1 * matches[student['id'], team]
                     for student in all_students
                     for team in all_teams)

    for student in all_students:
        solver.Add(solver.Sum([matches[student['id'], team] for team in all_teams]) == 1)
    print("`One team per student` added.")

    # Between MIN_TEAM_SIZE and MAX_TEAM_SIZE students per project
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] for student in all_students]) >= min_team_size)
        solver.Add(solver.Sum([matches[student['id'], team] for student in all_students]) <= max_team_size)
    print("`Team size added.`")


    #  1 < MBA <= 2
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'MBA' else 0) for student in all_students]) <= 2)
        if len([s for s in student if student['program'] == 'MBA']) >= len(all_teams):
            solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'MBA' else 0) for student in all_students]) >= 1)
        else:
            print("Not enough MBAs for have one per team.")

    #  LLM <= 1
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'LLM' else 0) for student in all_students]) <= 1)

    #  CS <= 2
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'CS' else 0) for student in all_students]) <= 2)

    # ORIE <= 1
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'ORIE' else 0) for student in all_students]) <= 1)

    # Jacobs <= 2
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] in ['CM', 'HT'] else 0) for student in all_students]) <= 2)

    objective = solver.Minimize(z)
    solver.Solve()
    s = solver.Objective().Value()
    print(s)
    
    buckets = {}
    for team in all_teams:
        buckets[team] = []

    for student in all_students:
        for team in all_teams:
            if int(matches[student['id'], team].SolutionValue()):
                buckets[team].append(student)
                
    return (all_teams, buckets)


# Another method to generate a different kind of team assignment.
def create_teams_homework(all_students, min_team_size, max_team_size):
    number_of_teams = math.floor(len(all_students) / min_team_size)
    all_teams = [ 'Team%03d' % k for k in range (1, number_of_teams+1)]
    
    solver = pywraplp.Solver('StudentProjectGridCBC', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    matches = {}
    for student in all_students:
        for team in all_teams:
            matches[student['id'], team] = solver.IntVar(0, 1, 'matches[%s,%s]' % (student['id'], team))

    z = solver.Sum(  1 * matches[student['id'], team]
                     for student in all_students
                     for team in all_teams)

    for student in all_students:
        solver.Add(solver.Sum([matches[student['id'], team] for team in all_teams]) == 1)
    print("`One team per student` added.")

    # Between MIN_TEAM_SIZE and MAX_TEAM_SIZE students per project
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] for student in all_students]) >= min_team_size)
        solver.Add(solver.Sum([matches[student['id'], team] for student in all_students]) <= max_team_size)
    print("`Team size added.`")


    #  MBA <= 1
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'MBA' else 0) for student in all_students]) <= 1)
        if len([s for s in student if student['program'] == 'MBA']) >= len(all_teams):
            solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'MBA' else 0) for student in all_students]) >= 1)
        else:
            print("Not enough MBAs for have one per team.")

    #  LLM <= 1
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'LLM' else 0) for student in all_students]) <= 1)

    #  CS <= 1
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'CS' else 0) for student in all_students]) <= 1)

    # ORIE <= 1
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'ORIE' else 0) for student in all_students]) <= 1)

    # Jacobs <= 2
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] in ['CM', 'HT'] else 0) for student in all_students]) <= 2)

    # Parsons <= 1
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'Parsons' else 0) for student in all_students]) <= 1)

    # No more than 2 people from same section
    for team in all_teams:
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['section'] == 'Section 1' else 0) for student in all_students]) <= 2)
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['section'] == 'Section 2' else 0) for student in all_students]) <= 2)
        solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['section'] == 'Section 3' else 0) for student in all_students]) <= 2)
    
    
    objective = solver.Minimize(z)
    solver.Solve()
    s = solver.Objective().Value()
    print(s)
    
    buckets = {}
    for team in all_teams:
        buckets[team] = []

    for student in all_students:
        for team in all_teams:
            if int(matches[student['id'], team].SolutionValue()):
                buckets[team].append(student)
                
    return (all_teams, buckets)

In [ ]:
def display_teams(teams, buckets, note="N/A"):
    for index, team in enumerate(teams):
        print("%s (%d people)" % (team, len(buckets[team])))
        print('-' * 20)
        for student in buckets[team]:
            email = student['id'] if '@' in student['id'] else "%s@cornell.edu" % student['id']
            print('"%s %s",%s,%s,%s,%s,%s' % (student['first_name'], student['last_name'], email, student['program'], team, student['section'], note))
        print()

def write_csv(writer, teams, buckets, note="N/A"):
    for index, team in enumerate(teams):
        writer.writerow(["%s (%d people)" % (team, len(buckets[team]))])
        for student in buckets[team]:
            writer.writerow([student['first_name'], student['last_name'], student['id'], student['program'], team, student['section'], note])
        writer.writerow([''])

In [ ]:
# We run the team assignment for TABLES.
CSV_FILENAME_PREFIX = "product-ideas-2018-tables-NOV"
for week in range(1, 8):
    with open(CSV_FILENAME_PREFIX + "-class%d.csv" % week, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for section in ['Section 1', 'Section 2', 'Section 3']:
            students = [s for s in ALL_STUDENTS if s['section'] == section]
            random.shuffle(students)
            (teams, buckets) = create_teams(students, 5, 6)
            display_teams(teams, buckets, "Week %d" % week)
            write_csv(writer, teams, buckets, "Week %d" % week)

In [ ]:
# We run the team assignment for HOMEWORKS.
CSV_FILENAME_PREFIX = "product-ideas-2018-homeworks-NOV"
for week in range(1, 10):
    with open(CSV_FILENAME_PREFIX + "-class%d.csv" % week, 'w') as csvfile:
        writer = csv.writer(csvfile)
        students = [s for s in ALL_STUDENTS]
        print(len(students))
        random.shuffle(students)
        (teams, buckets) = create_teams_homework(students, 3, 4)
        display_teams(teams, buckets, "Week %d" % week)
        write_csv(writer, teams, buckets, "Week %d" % week)